In [1]:
from sqlanalyzer import column_parser, unbundle
import sqlparse
import re
import json
import pandas as pd
import time


def flatten_subquery(final_list, sub_queries, level_num):
    
    for q in sub_queries:
        for alias,query in q.items():
            formatter = column_parser.Parser(query)
            formatted_query = formatter.format_query(query)
            unbundled = unbundle.Unbundle(formatted_query)
            query_dict = {}
            if unbundled.has_child(query):
                if alias == 'no alias' or alias == '' or alias == 'query':
                    query_dict, sub_queries = unbundled.restructure_subquery(query_dict, 'level_{}_main'.format(level_num), formatted_query)
                else:
                    query_dict, sub_queries = unbundled.restructure_subquery(query_dict, alias, formatted_query)
            else: 
                sub_queries = []

        if query_dict != {}:
            final_list.append(query_dict)

        for subq in sub_queries:
            for _, sub_query in subq.items():
                if not unbundled.has_child(sub_query): 
                    final_list.append(subq)
                    sub_queries.remove(subq)

    return final_list, sub_queries


def is_cte(query):
    return query.startswith('WITH')


def flatten_pure_nested(query):

    sub_queries = [{'query': query}]
    final_list = []
    i = 0

    while sub_queries != []:
        i += 1
        final_list, sub_queries = flatten_subquery(final_list, sub_queries, level_num=i)

    return final_list

## type A: subquery ( subqueries ) 

SELECT FROM (SELECT FROM (...))

In [2]:
query = """SELECT *
   FROM
     (SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
      LEFT JOIN
        (SELECT id,
                email,
                created
         FROM mapbox_customer_data.accounts
         WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id
      LEFT JOIN
        (SELECT anonymous_id AS anon_id_ad,
                context_campaign_name,
                min(TIMESTAMP) AS min_exposure
         FROM mapbox_customer_data.segment_pages
         WHERE dt >= '2018-07-01'
           AND context_campaign_name IS NOT NULL
         GROUP BY 1,
                  2) c ON a.anonymous_id = c.anon_id_ad
      LEFT JOIN
        (SELECT DISTINCT anonymous_id AS anon_id_event,
                         original_timestamp,
                         event,
                         context_traits_email
         FROM mapbox_customer_data.segment_tracks
         WHERE dt >= '2018-07-01'
           AND event LIKE 'submitted_%form'
           AND context_traits_email IS NOT NULL ) d ON a.anonymous_id = d.anon_id_event
    LEFT JOIN
        (SELECT sfdc_accounts.platform, sfdc_accounts.mobile_os, sfdc_accounts.service_metadata,
sfdc_cases.account, sfdc_cases.num_requests, sfdc_cases.owner, sfdc_accounts.user_id
FROM sfdc.accounts sfdc_accounts
LEFT JOIN 
(SELECT MAX(dt) FROM 
    (SELECT dt 
    FROM sfdc.oppty 
    LEFT JOIN (SELECT MAX(dt) FROM (SELECT DISTINCT dt FROM sfdc.owner AS sfdc_owner) AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt)
    LEFT JOIN (SELECT dt FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) )
AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt
LEFT JOIN sfdc.cases AS sfdc_cases ON sfdc_cases.id = sfdc_accounts.case_id
WHERE sfdc_cases_oppty.dt > '2020-04-03' AND sfdc_cases_oppty.dt < '2020-05-04' ORDER BY 1 GROUP BY 3 LIMIT 20
        ) e ON e.user_id = a.user_id
        )
   WHERE context_campaign_name IS NOT NULL 
"""


In [3]:
flatten_pure_nested(query)

[{'level_1_main': 'SELECT * WHERE context_campaign_name IS NOT NULL FROM no alias '},
 {'level_2_main': 'SELECT a.*,        b.*,        c.*,        d.* FROM a LEFT JOIN b ON a.user_id = b.id LEFT JOIN c ON a.anonymous_id = c.anon_id_ad LEFT JOIN d ON a.anonymous_id = d.anon_id_event LEFT JOIN e ON e.user_id = a.user_id '},
 {'a': "SELECT DISTINCT anonymous_id, user_id FROM mapbox_customer_data.segment_identifies WHERE dt >= '2018-07-01' AND anonymous_id IS NOT NULL AND user_id IS NOT NULL "},
 {'c': "SELECT anonymous_id AS anon_id_ad, context_campaign_name, min(TIMESTAMP) AS min_exposure FROM mapbox_customer_data.segment_pages WHERE dt >= '2018-07-01' AND context_campaign_name IS NOT NULL GROUP BY 1, 2"},
 {'e': "SELECT sfdc_accounts.platform,        sfdc_accounts.mobile_os,        sfdc_accounts.service_metadata,        sfdc_cases.account,        sfdc_cases.num_requests,        sfdc_cases.owner,        sfdc_accounts.user_id WHERE sfdc_cases_oppty.dt > '2020-04-03'   AND sfdc_cases_oppt

## type B: CTE ( subqueries )

WITH a AS ()...

In [4]:
query = """WITH
    -- day_dummy AS -- PRESTO: need to make daily dummy dates in qubole notebook
    -- (
    --   SELECT
    --   'dummy' AS dummy,
    --   CAST(CAST(date_column AS DATE) AS VARCHAR) AS dt
    --   FROM
    --   (
    --     VALUES
    --       (SEQUENCE(FROM_ISO8601_DATE('2018-11-01'),
    --                 FROM_ISO8601_DATE('2022-01-01'),
    --                 INTERVAL '1' DAY)
    --       )
    --   ) AS t1(date_array)
    --   CROSS JOIN UNNEST(date_array) AS t2(date_column)
    -- ),
    day_dummy AS -- HIVE: need to make daily dummy dates in qubole notebook
    (
      SELECT
      'dummy' AS dummy,
      DATE_ADD(t.product_start_day, pe.i) AS dt
      FROM
      (
        SELECT
        MIN('2018-11-01') AS product_start_day,
        MAX('2022-01-01') AS product_end_day
        FROM
        analytics.service_endpoint_mapping -- dummy table
        WHERE
        dt = '2020-02-06'
      ) t
      LATERAL VIEW POSEXPLODE(SPLIT(SPACE(DATEDIFF(t.product_end_day, t.product_start_day)),' ')) pe as i,x
    ),
    apa AS
    (
      SELECT -- select all opportunities that are won or 7-ICR APA
      o.id AS apa_opportunity_id,
      o.mapbox_username_c AS account,
      CAST(service_date AS DATE) AS apa_start_date,
      CAST(end_date_c AS DATE) AS apa_end_date
      FROM
      sfdc.opportunities o
      INNER JOIN
      sfdc.opportunity_product p
      ON
      -- o.id = p.opportunity_id AND p.dt = (SELECT MAX(dt) FROM sfdc.opportunity_product)
      o.id = p.opportunity_id
      WHERE
      p.dt = (SELECT MAX(dt) FROM sfdc.opportunity_product)
      AND
      o.dt = (SELECT MAX(dt) FROM sfdc.opportunities)
      AND 
      product_2_id = '01t1R000005tfSqQAI' --APA product id
      AND
      -- stage_name IN ('Won', '7 - ICR')
      stage_name IN ('Won')
    ),
    paygo_apa AS
    (
      SELECT
      o.*,
      CASE WHEN a.account IS NOT NULL THEN 'apa' ELSE 'non_apa' END AS invoice_type
      FROM
      wbr.paygo_by_service o
      LEFT JOIN
      apa a
      ON
      o.account = a.account AND DATE_TRUNC('month', CAST(o.invoice_dt AS DATE)) BETWEEN DATE_TRUNC('month', a.apa_start_date) + INTERVAL '1' MONTH AND DATE_TRUNC('month', a.apa_end_date) + INTERVAL '1' MONTH
      WHERE
      dt = (SELECT MAX(dt) FROM wbr.paygo_by_service)
    ),
    paygo_tmp_1 AS -- service_metadata
    (
      SELECT
      dt AS snapshot_dt,
      invoice_dt AS dt,
      -- account,
      payment_status,
      invoice_type,
      service,
      SUM(item_amount) AS amount
      FROM
      paygo_apa
      WHERE
      is_prepay_item = false
      AND
      payment_status IN ('paid', 'outstanding')
      AND
      CAST(invoice_dt AS DATE) >= CAST('2018-12-01' AS DATE) - INTERVAL '29' DAY
      GROUP BY
      1,2,3,4,5
      UNION ALL
      SELECT
      dt AS snapshot_dt,
      invoice_dt AS dt,
      -- account,
      payment_status,
      invoice_type,
      '_all' AS service,
      SUM(item_amount) AS amount
      FROM
      paygo_apa
      WHERE
      is_prepay_item = false
      AND
      payment_status IN ('paid', 'outstanding')
      AND
      CAST(invoice_dt AS DATE) >= CAST('2018-12-01' AS DATE) - INTERVAL '29' DAY
      GROUP BY
      1,2,3,4,5
      UNION ALL
      SELECT
      dt AS snapshot_dt,
      invoice_dt AS dt,
      -- account,
      payment_status,
      '_all' AS invoice_type,
      '_all' AS service,
      SUM(item_amount) AS amount
      FROM
      paygo_apa
      WHERE
      is_prepay_item = false
      AND
      payment_status IN ('paid', 'outstanding')
      AND
      CAST(invoice_dt AS DATE) >= CAST('2018-12-01' AS DATE) - INTERVAL '29' DAY
      GROUP BY
      1,2,3,4,5
    ),
    paygo_tmp AS
    (
      SELECT
      b.dt,
      a.snapshot_dt,
      a.payment_status,
      a.invoice_type,
      a.service,
      COALESCE(c.amount, 0) AS amount
      FROM
      (
        SELECT
        'dummy' AS dummy,
        snapshot_dt,
        payment_status,
        invoice_type,
        service,
        MIN(CAST(dt AS DATE)) AS min_dt,
        MAX(CAST(dt AS DATE) + INTERVAL '29' DAY) AS max_dt
        FROM
        paygo_tmp_1
        GROUP BY
        1,2,3,4,5
      ) a
      INNER JOIN
      day_dummy b
      ON
      a.dummy = b.dummy AND CAST(b.dt AS DATE) BETWEEN a.min_dt AND a.max_dt
      LEFT JOIN
      paygo_tmp_1 c
      ON
      b.dt = c.dt AND a.payment_status = c.payment_status AND a.service = c.service AND a.invoice_type = c.invoice_type
    ),
    paygo AS -- WINDOW
    (
      SELECT
      dt,
      payment_status,
      invoice_type,
      aggregation,
      service,
      value
      FROM
      (
        SELECT
        dt,
        snapshot_dt,
        payment_status,
        invoice_type,
        '7d' AS aggregation,
        service,
        SUM(amount) OVER (PARTITION BY payment_status, invoice_type, service ORDER BY dt ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS value
        FROM
        paygo_tmp
        UNION ALL
        SELECT
        dt,
        snapshot_dt,
        payment_status,
        invoice_type,
        '30d' AS aggregation,
        service,
        SUM(amount) OVER (PARTITION BY payment_status, invoice_type, service ORDER BY dt ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS value
        FROM
        paygo_tmp
        UNION ALL
        SELECT
        dt,
        snapshot_dt,
        payment_status,
        invoice_type,
        '1d' AS aggregation,
        service,
        amount AS value
        FROM
        paygo_tmp
      ) a
      WHERE
      dt BETWEEN '2018-12-01' AND snapshot_dt
    ),
    main AS
    (
      SELECT
      dt,
      'paygo_billed_paid_rev' AS metric, 
      aggregation,
      service,
      value
      FROM
      paygo
      WHERE
      payment_status = 'paid'
      AND
      invoice_type = '_all'
      UNION ALL
      SELECT
      dt,
      'paygo_billed_unpaid_rev' AS metric,
      aggregation,
      service,
      value
      FROM
      paygo
      WHERE
      payment_status = 'outstanding'
      AND
      invoice_type = '_all'
      UNION ALL
      SELECT
      dt,
      'paygo_nonapa_paid_rev' AS metric, 
      aggregation,
      service,
      value
      FROM
      paygo
      WHERE
      payment_status = 'paid'
      AND
      invoice_type = 'non_apa'
      UNION ALL
      SELECT
      dt,
      'paygo_nonapa_unpaid_rev' AS metric,
      aggregation,
      service,
      value
      FROM
      paygo
      WHERE
      payment_status = 'outstanding'
      AND
      invoice_type = 'non_apa'
      UNION ALL
      SELECT
      dt,
      'paygo_apa_paid_rev' AS metric,
      aggregation,
      service,
      value
      FROM
      paygo
      WHERE
      payment_status = 'paid'
      AND
      invoice_type = 'apa'
      UNION ALL
      SELECT
      dt,
      'paygo_apa_unpaid_rev' AS metric,
      aggregation,
      service,
      value
      FROM
      paygo
      WHERE
      payment_status = 'outstanding'
      AND
      invoice_type = 'apa'
    )
    SELECT
    CAST(dt AS STRING) AS event_dt,
    metric,
    aggregation,
    service,
    value
    FROM
    main
"""

In [5]:
if query.startswith('WITH'):
    formatter = column_parser.Parser(query)
    formatted_query = formatter.format_query(query)
    query_list = formatted_query.split('\n')

In [6]:
cte_dict = formatter.parse_cte(formatted_query)
# cte_dict

In [7]:
unbundled = unbundle.Unbundle(formatted_query)
final_list = []
for alias, cte_query in cte_dict.items():
    if unbundled.has_child(cte_query):
        final_list.append({alias: flatten_pure_nested(cte_query)})
    else:
        final_list.append({alias: cte_query})
    

In [8]:
final_list

[{'day_dummy': [{'level_1_main': "SELECT 'dummy' AS dummy,        DATE_ADD(t.product_start_day, pe.i) AS dt FROM i "},
   {'i': "SELECT MIN('2018-11-01') AS product_start_day, MAX('2022-01-01') AS product_end_day FROM analytics.service_endpoint_mapping WHERE dt = '2020-02-06' ) t LATERAL VIEW POSEXPLODE(SPLIT(SPACE(DATEDIFF(t.product_end_day, t.product_start_day)), ' ')) pe"}]},
 {'x), apa': [{'level_1_main': "SELECT o.id AS apa_opportunity_id,        o.mapbox_username_c AS account,        CAST(service_date AS DATE) AS apa_start_date,        CAST(end_date_c AS DATE) AS apa_end_date WHERE p.dt =     (SELECT MAX(dt)      FROM sfdc.opportunity_product)   AND o.dt =     (SELECT MAX(dt)      FROM sfdc.opportunities)   AND product_2_id = '01t1R000005tfSqQAI'   AND stage_name IN ('Won') FROM sfdc.opportunities o INNER JOIN sfdc.opportunity_product p ON o.id = p.opportunity_id "}]},
 {'paygo_apa': [{'level_1_main': "SELECT o.*,        CASE            WHEN a.account IS NOT NULL THEN 'apa'      

In [9]:
cte_dict

{'day_dummy': "SELECT 'dummy' AS dummy,\n          DATE_ADD(t.product_start_day, pe.i) AS dt\n   FROM\n     (SELECT MIN('2018-11-01') AS product_start_day,\n             MAX('2022-01-01') AS product_end_day\n      FROM analytics.service_endpoint_mapping\n      WHERE dt = '2020-02-06' ) t LATERAL VIEW POSEXPLODE(SPLIT(SPACE(DATEDIFF(t.product_end_day, t.product_start_day)), ' ')) pe AS i",
 'x), apa': "SELECT o.id AS apa_opportunity_id,\n          o.mapbox_username_c AS account,\n          CAST(service_date AS DATE) AS apa_start_date,\n          CAST(end_date_c AS DATE) AS apa_end_date\n   FROM sfdc.opportunities o\n   INNER JOIN sfdc.opportunity_product p ON o.id = p.opportunity_id\n   WHERE p.dt =\n       (SELECT MAX(dt)\n        FROM sfdc.opportunity_product)\n     AND o.dt =\n       (SELECT MAX(dt)\n        FROM sfdc.opportunities)\n     AND product_2_id = '01t1R000005tfSqQAI'\n     AND stage_name IN ('Won') ",
 'paygo_apa': "SELECT o.*,\n          CASE\n              WHEN a.account

## type C: subquery ( CTE ( subqueries ) )

SELECT FROM (SELECT FROM (WITH a AS ()...))

In [ ]:
query = """SELECT * FROM (SELECT *\nFROM (\n  with reg_users as (\n  \n  \tselect * \n  \tfrom (\n  \tselect \n  \t        a.*\n  \t        , b.*\n  \t        , c.*\n  \t        , d.*\n  \tfrom \n  \t(\n  \tselect \n  \t        distinct \n  \t        anonymous_id\n  \t        , user_id\n  \tfrom mapbox_customer_data.segment_identifies\n  \twhere dt >= \'2018-07-01\'\n  \tand anonymous_id is not null\n  \tand user_id is not null\n  \t) a\n  \n  \tleft join \n  \n  \t(\n  \tselect \n  \t        id \n  \t        , email\n  \t        , created\n  \tfrom mapbox_customer_data.accounts\n  \twhere cast(dt as DATE) = CURRENT_DATE - INTERVAL \'1\' DAY \n  \t) b\n  \t        on a.user_id = b.id\n  \n  \tleft join \n  \n  \t(\n  \t    \n  \t        select        \n  \t                anonymous_id as anon_id_ad\n  \t                , context_campaign_name\n  \t                , min(timestamp) as min_exposure\n  \t        from mapbox_customer_data.segment_pages\n  \t        where dt >= \'2018-07-01\'\n  \t        and context_campaign_name is not null\n  \t        group by 1,2\n  \n  \t) c \n  \t        on a.anonymous_id = c.anon_id_ad\n  \t        \n  \tleft join \n  \n  \t(\n  \t        select \n  \t                distinct\n  \t                anonymous_id as anon_id_event\n  \t                , original_timestamp\n  \t                , event\n  \t                , context_traits_email\n  \t        from mapbox_customer_data.segment_tracks\n  \t        where dt >= \'2018-07-01\'\n  \t        and event like \'submitted_%form\'\n  \t        and context_traits_email is not null\n  \t) d\n  \t        on a.anonymous_id = d.anon_id_event\n  \n  \t) \n  \twhere context_campaign_name is not null\n  \n  ), \n  \n  non_reg_users as (\n  \n  \tselect \n  \t        context_campaign_name\n  \t        , min_exposure\n  \t        , event\n  \t        , original_timestamp as event_timestamp\n  \t        , context_traits_email as event_email\n  \tfrom (\n  \tselect a.*\n  \t        , b.*\n  \tfrom \n  \t(\n  \t        select \n  \t                anonymous_id as anon_id_ad\n  \t                , context_campaign_name\n  \t                , min(original_timestamp) as min_exposure\n  \t        from (       \n  \t        select        \n  \t                context_campaign_name\n  \t                , anonymous_id\n  \t                , original_timestamp \n  \t        from mapbox_customer_data.segment_pages\n  \t        where dt >= \'2018-07-01\'\n  \t        and context_campaign_name is not null\n  \t        )\n  \t        group by 1,2\n  \t) a\n  \n  \tleft join \n  \t(\n  \t        select \n  \t                distinct\n  \t                anonymous_id as anon_id_event\n  \t                , original_timestamp\n  \t                , event\n  \t                , context_traits_email\n  \t        from mapbox_customer_data.segment_tracks\n  \t        where dt >= \'2018-07-01\'\n  \t        and event like \'submitted_%form\'\n  \t        and context_traits_email is not null\n  \t) b\n  \t        on a.anon_id_ad = b.anon_id_event\n  \n  \t)\n  \twhere anon_id_event is not null\n  \tand to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n  \tand cast(min_exposure as DATE) >= cast(original_timestamp as DATE) - INTERVAL \'28\' DAY\n  \n  \n  ), \n  \n  mql_flag as (\n  \n  \tselect \n  \t        email\n  \t        , created_date\n  \t        , last_mql_date_c\n  \t        , mql_flag\n  \tfrom (\n  \n  \tselect \n  \t        email\n  \t        , min(created_date) created_date\n  \t        , max(last_mql_date_c) last_mql_date_c\n  \t        , case when max(last_mql_date_c) is not null then 1 else 0 end as mql_flag\n  \t        , sum(case when is_deleted = true then 1 else 0 end) as is_deleted\n  \tfrom sales.salesforce_leads\n  \twhere cast(dt as DATE) = CURRENT_DATE - INTERVAL \'1\' DAY \n  \tgroup by 1\n  \t)\n  \twhere mql_flag = 1\n  \tand is_deleted = 0\n  \n  ),\n  \n  cleaned_list as (\n  \n  \n  \tselect \n  \t\tdistinct \n  \t\t\t*\n  \tfrom (\n  \t\tselect \n  \t\t        context_campaign_name\n  \t\t        , min_exposure\n  \t\t        , \'created_an_account\' as event\n  \t\t        , created as event_timestamp\n  \t\t        , email as event_email\n  \t\tfrom reg_users        \n  \t\twhere to_unixtime(min_exposure) <= to_unixtime(created)\n  \t\tand cast(min_exposure as DATE) >= cast(created as DATE) - INTERVAL \'28\' DAY\n  \n  \n  \t\tunion all\n  \n  \n  \t\tselect \n  \t\t        context_campaign_name\n  \t\t        , min_exposure\n  \t\t        , event\n  \t\t        , original_timestamp as event_timestamp\n  \t\t        , context_traits_email as event_email\n  \t\tfrom reg_users      \n  \t\twhere to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n  \t\tand cast(min_exposure as DATE) >= cast(original_timestamp as DATE) - INTERVAL \'28\' DAY\n  \n  \t\tunion all\n  \n  \t\tselect * \n  \t\tfrom non_reg_users\n  \t)\n  \n  )\n  \n  \n  \n  select \n  \ta.*\n  \t, b.*\n  from cleaned_list a\n  left join mql_flag b \n  \ton a.event_email = b.email\n) "custom_sql_query"\nLIMIT 0) T LIMIT 0
"""
